Importing Libraries:



In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import *
from keras.optimizers import Adam
from keras.models import Sequential
from keras import Model

Loading the Images into the Notebook:

In [2]:
#Reading all the images into a numpy array
images = []
for file in os.listdir():
  if file.endswith('.jpg'):
    #if the file is an image
    image_values = cv2.imread(file)

    #resize the image into 360 x 480 pixels
    image_values = cv2.resize(image_values,(360,480))

    images.append(image_values)

#storing everything in a numpy array
images_array = np.array(images)

Knowing the number of Images we are working on:

In [3]:
number_of_observations = len(images_array)
number_of_observations

31

We don't have any columns because we're operating on an image dataset.

Analyzing image storage in Python variables.

In [4]:
images_array

array([[[[ 23,  36,  38],
         [ 23,  36,  38],
         [ 23,  36,  38],
         ...,
         [ 16,  24,  24],
         [ 16,  24,  24],
         [ 16,  24,  24]],

        [[ 23,  36,  38],
         [ 23,  36,  38],
         [ 23,  36,  38],
         ...,
         [ 16,  24,  24],
         [ 16,  24,  24],
         [ 16,  24,  24]],

        [[ 23,  36,  38],
         [ 23,  36,  38],
         [ 23,  36,  38],
         ...,
         [ 16,  24,  24],
         [ 16,  24,  24],
         [ 16,  24,  24]],

        ...,

        [[ 24,  41,  69],
         [ 24,  42,  70],
         [ 25,  44,  71],
         ...,
         [101, 110, 130],
         [101, 110, 130],
         [101, 110, 130]],

        [[ 29,  47,  75],
         [ 29,  48,  76],
         [ 30,  50,  77],
         ...,
         [101, 110, 130],
         [101, 110, 130],
         [101, 110, 130]],

        [[ 31,  49,  78],
         [ 31,  50,  78],
         [ 32,  52,  80],
         ...,
         [101, 110, 130],
        

In [5]:
shape = images_array[0].shape
shape

(480, 360, 3)

In [6]:
images_array[0]

array([[[ 23,  36,  38],
        [ 23,  36,  38],
        [ 23,  36,  38],
        ...,
        [ 16,  24,  24],
        [ 16,  24,  24],
        [ 16,  24,  24]],

       [[ 23,  36,  38],
        [ 23,  36,  38],
        [ 23,  36,  38],
        ...,
        [ 16,  24,  24],
        [ 16,  24,  24],
        [ 16,  24,  24]],

       [[ 23,  36,  38],
        [ 23,  36,  38],
        [ 23,  36,  38],
        ...,
        [ 16,  24,  24],
        [ 16,  24,  24],
        [ 16,  24,  24]],

       ...,

       [[ 24,  41,  69],
        [ 24,  42,  70],
        [ 25,  44,  71],
        ...,
        [101, 110, 130],
        [101, 110, 130],
        [101, 110, 130]],

       [[ 29,  47,  75],
        [ 29,  48,  76],
        [ 30,  50,  77],
        ...,
        [101, 110, 130],
        [101, 110, 130],
        [101, 110, 130]],

       [[ 31,  49,  78],
        [ 31,  50,  78],
        [ 32,  52,  80],
        ...,
        [101, 110, 130],
        [101, 110, 130],
        [101, 110, 130]]

We can see that the images are stored pixel by pixel as a two - dimensional array, with each element being an array with three parameters. These three parameters are the Pixel's RGB (i.e. Red, Green and Blue) values.

There aren't any values that are missing.

All of the pixel specifics in the image dataset are utilized as characteristics in the construction of our machine learning model.

Intended column: Our model will generate new facial images based on the images we feed it. We as well train a discriminator to determine if an image is genuine or synthetic.

Actions to be Implemented in the project:

1. The dataset will be loaded into numpy arrays.
2. We will pre - process the images in order to fit their pixel density.
3. We'll build a GAN model and train it on the images we provide it..
4. We will build a discriminator to determine if an image is genuine or synthetic.
5. Hyper - parameters optimizations will be carried out.
6. By using generator, we would therefore generate new images.
7. We then display the synthetically generated images through a User Interface.

In [7]:
def build_discriminator(img_shape):
  model = Sequential([                                                      
  Conv2D(32, kernel_size=5, strides=2, input_shape=img_shape, padding="same"),                          
  LeakyReLU(alpha=0.2),                           
  Dropout(0.25),
  Conv2D(64, kernel_size=5, strides=2, padding="same"),                          
  BatchNormalization(momentum=0.8),                           
  LeakyReLU(alpha=0.2),                           
  Dropout(0.25),
  Conv2D(128, kernel_size=5, strides=2, padding="same"),                         
  BatchNormalization(momentum=0.8),                           
  LeakyReLU(alpha=0.2),                           
  Dropout(0.25),
  Flatten(),                            
  Dense(1),                           
  Activation("sigmoid")
  ])
                                                    
  model.summary()                           
  img = Input(shape=img_shape)                           
  d_pred = model(img)                           
  return Model(inputs=img, outputs=d_pred)

In [8]:
def build_generator(z_dimension, channels):                           
  model = Sequential([
  Dense(2 * 120 * 90, input_dim=z_dimension),                           
  LeakyReLU(alpha=0.2),                            
  Reshape((120, 90, 2)),
  UpSampling2D(),                          
  Conv2D(128, kernel_size=5, padding="same"),                           
  BatchNormalization(momentum=0.8),                           
  LeakyReLU(alpha=0.2),
  UpSampling2D(),                           
  Conv2D(64, kernel_size=5, padding="same"),                           
  BatchNormalization(momentum=0.8),                           
  LeakyReLU(alpha=0.2),
  Conv2D(channels, kernel_size=5, padding="same"),                           
  Activation("tanh"),
  ])
  model.summary()                           
  noise = Input(shape=(z_dimension,))                           
  img = model(noise)                           
  return Model(inputs=noise, outputs=img)

In [9]:
#load real pictures:                       
x_train = images_array                                       
# model parameters                       
img_rows = shape[0]                       
img_cols = shape[1]                       
channels = 3                       
img_shape = (img_rows, img_cols, channels)                       
z_dimension = 32                       
optimizer = Adam(0.0005, 0.5)

In [10]:
discriminator = build_discriminator(img_shape)                       
discriminator.compile(loss='binary_crossentropy',                                             
                      optimizer=optimizer, metrics=['accuracy'])                                               
generator = build_generator(z_dimension,channels)                                               
z = Input(shape=(z_dimension,))                       
img = generator(z)                       
discriminator.trainable = False                       
d_pred = discriminator(img)                       
combined = Model(z, d_pred)                       
combined.compile(loss='binary_crossentropy',optimizer=optimizer,                                        
                 metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 240, 180, 32)      2432      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 240, 180, 32)      0         
                                                                 
 dropout (Dropout)           (None, 240, 180, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 120, 90, 64)       51264     
                                                                 
 batch_normalization (BatchN  (None, 120, 90, 64)      256       
 ormalization)                                                   
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 120, 90, 64)       0         
                                                        

In [ ]:
max_epochs = 30000
batch = 64

real_imgs = np.ones((batch, 1))
fake_imgs = np.zeros((batch, 1))

for epoch in range(max_epochs):
    index = np.random.randint(0, x_train.shape[0], batch)
    images = x_train[index]
    sample_noise = np.random.normal(0, 1, (batch, z_dimension))
    gen_images = generator.predict(sample_noise)

    fake_imgs_discriminator_loss = discriminator.train_on_batch(gen_images, fake_imgs)
    real_imgs_discriminator_loss = discriminator.train_on_batch(images, real_imgs)
    
    total_discriminator_loss = np.add(real_imgs_discriminator_loss, fake_imgs_discriminator_loss)/2
    sample_noise = np.random.normal(0, 1, (batch, z_dimension))
    g_loss = combined.train_on_batch(sample_noise, real_imgs)
    
generator.save("generator.h5" )

2/2 [==============================] - 103s 51s/step
